In [1]:
import os
#import leafmap
import folium
import torch
from samgeo import SamGeo, tms_to_geotiff
from shapely.geometry import Polygon
import geopandas as gpd

In [2]:
#bbox = [14.27604402,  9.28422419, 16.36691269, 11.94281461]#[-95.3704, 29.6762, -95.368, 29.6775]
#bbox= [15.097455,9.8055,15.1010,9.8079]
bbox= [15.094455,9.8255,15.1610,9.9279]
polygon_coords = [
        [bbox[1], bbox[0]],
        [bbox[1], bbox[2]],
        [bbox[3], bbox[2]],
        [bbox[3], bbox[0]],
        [bbox[1], bbox[0]],
    ]
center = Polygon(polygon_coords).centroid.coords[0]

In [3]:
m = folium.Map(location=center, zoom_start=18)
# Add the Google Maps basemap to the map
#folium.TileLayer('https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', attr='Google Maps', name='Hybrid Imagery').add_to(m)
#folium.TileLayer('https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', attr='Google Maps', name='Hybrid Imagery').add_to(m)
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='Esri World Imagery', name='Satellite Imagery').add_to(m)
# Add the bounding box to the map as a rectangle
folium.Polygon(
    polygon_coords,
    color='red',
    #fill_color='red',
    fill_opacity=0.2
).add_to(m)

# Create a layer control and add it to the map
folium.LayerControl().add_to(m)
# Display the map
#m.add_basemap('SATELLITE')
m

In [ ]:
# if m.user_roi_bounds() is not None:
#     bbox = m.user_roi_bounds()
# else:
#     bbox = [-95.3704, 29.6762, -95.368, 29.6775]

In [10]:
image = '/Documents/river.tif'
# image = '/path/to/your/own/image.tif'

In [ ]:
source = 'Satellite'
#source = https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}
tms_to_geotiff(output=image, bbox=bbox, zoom=18, source= source, overwrite=True, quiet=True)

In [5]:
checkpoint = r'C:\Users\oscar.bautista\Downloads\SAM\sam_vit_h_4b8939.pth'

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

In [11]:
mask = 'Gabra_segment.tiff'
sam.generate(image, mask)

RasterioIOError: '/Documents/river.tif' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
shapefile = r'C:\Geotar\GLOBAL\geodata\workspace\GEOSAM\chad_south_segment.shp'
sam.tiff_to_vector(mask, shapefile)

In [ ]:
import folium
import geopandas as gpd

# Create a new map centered at a specific location
map = folium.Map(location=center, zoom_start=18)

# Read the shapefile using geopandas
gdf = gpd.read_file(shapefile)

# Reproject the GeoDataFrame to EPSG:4326
#gdf = gdf.to_crs('EPSG:4326')

# Add the GeoDataFrame as a GeoJSON layer
folium.GeoJson(
    gdf,
    name='Shapefile'
).add_to(map)

# Add Google Imagery base map
#folium.TileLayer('https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', name='Google Imagery', attr='Google').add_to(map)

# Add Esri Imagery base map
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='Esri World Imagery', name='Satellite Imagery').add_to(map)


# Add Layer Control to toggle layers
folium.LayerControl().add_to(map)

# Display the map
map
